In [8]:
# !pip3 install langchain
# !pip3 install openai
# !pip3 install chromadb
# !pip3 install pypdf

In [12]:
from langchain.document_loaders import PyPDFLoader 
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain 
from langchain.llms import OpenAI 
import os
import json

with open("config.json", "r+") as f : 
    config = json.load(f)
f.close()

os.environ["OPENAI_API_KEY"] = config['API_Keys']['OpenAI']

In [29]:
pdf_paths = "/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/" 
pdfs = [os.path.join(pdf_paths, f) for f in os.listdir(pdf_paths) if f.endswith(".pdf")]
pdfs.sort()
pdfs

['/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase0.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase1.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase2.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase3.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase4.pdf',
 '/Users/nandy/Documents/GitHub/LLM_Research/Test_documents/AbbotData/Hcase5.pdf']

In [91]:
loader = PyPDFLoader(pdfs[5])
pages = loader.load_and_split()
print(pages[0].page_content)

Observed problemsObserved problems provide information about problems that may occur on the system andprovide corrective actions that help to resolve the problems.If the corrective actions for an observed problem do not resolve the problem, contact the localrepresentative or find country-specific contact information at transfusion.abbott.NOTE: Corrective actions may involve hazardous activity. Use caution to minimize operatorexposure and to prevent personal injury or system damage. Examples of hazardous activitiesinclude:• The replacement of system probes.• The use of reagents, calibrators, controls, and specimens.• The removal of physical obstructions.• The removal of system waste.Related information...Troubleshooting, page 601Instrument observed problems, page 886Sample results observed problems, page 892User interface (UI) computer observed problems, page 901Printer observed problems, page 902Instrument observed problemsThe instrument observed problems include problems that occur on

In [92]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pages, embeddings=embeddings, persist_directory=".") 

Using embedded DuckDB with persistence: data will be stored in: .
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [93]:
# import pickle
# pickle.dump(vectordb.persist, open("/Users/nandy/Documents/GitHub/LLM_Research/LangChain/Abbot_data/abbottchromadb.pkl", 'wb'))
# explore the vectordb object to see if we can store it locally, or in pinecone maybe 

In [94]:
pages[0].page_content

'Observed problemsObserved problems provide information about problems that may occur on the system andprovide corrective actions that help to resolve the problems.If the corrective actions for an observed problem do not resolve the problem, contact the localrepresentative or find country-specific contact information at transfusion.abbott.NOTE: Corrective actions may involve hazardous activity. Use caution to minimize operatorexposure and to prevent personal injury or system damage. Examples of hazardous activitiesinclude:• The replacement of system probes.• The use of reagents, calibrators, controls, and specimens.• The removal of physical obstructions.• The removal of system waste.Related information...Troubleshooting, page 601Instrument observed problems, page 886Sample results observed problems, page 892User interface (UI) computer observed problems, page 901Printer observed problems, page 902Instrument observed problemsThe instrument observed problems include problems that occur o

In [95]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
                                    vectordb.as_retriever(search_kwargs={"k": 1}), return_source_documents=True)

In [99]:
#turn one 
#Are there any open source or third-party software components included in the Alinity s System software?
chat_history = []
query = "What can be the reason for Optics Verification procedure failed ? "
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'There can be various reasons for Optics Verification procedure to fail, including but not limited to: condensation on the Optics Verifier cartridge label, frozen Optics Verifier solution, expired Optics Verifier solution, incorrect target RLU value entered, deteriorated Optics Verifier solution, and hardware failure. The specific reason for the failure would need to be determined by reviewing the message codes obtained during the failed procedure and performing the corrective action for those message codes.'

In [101]:
#turn two
chat_history.append((query, result["answer"]))
query = "What is the corrective action when condensation on the Optics Verifier cartridge label occur ? "
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'The corrective action for condensation on the Optics Verifier cartridge label during the Optics Verification procedure is to wipe the Optics Verifier cartridge label with a lint-free tissue to remove condensation.'

[('What should I do for a clean install?',
  'For a clean install, I should drop the Equip. CFG file into place and import the IP addresses. I may need to run the sl -Msg2Fld tool if necessary.')]

In [54]:
# continous exec
query = " "
chat_history = []
while (text != ""):
    query = input()
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    print(result['answer'])

 What's the name?
The tool that should be run for a clean install is the sl -Msg2Fld tool.
 Where should I go for it 
The tool that should be run for a clean install is the sl -Msg2Fld tool.
 
The tool that should be run for a clean install is the sl -Msg2Fld tool.


##### Stability Experiment